# Project Title
### Data Engineering Capstone Project

#### Project Summary
The project will be analysing immigration data in US and provide an insights into the various types of visa issued to immigrants as well as the demographics of immigrants. 

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import os 
import glob

import datetime
from pprint import pprint
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, weekofyear, date_format, isnull
from pyspark.sql.types import StringType, DateType , IntegerType


In [2]:
# Create a Spark session
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

### Step 1: Scope the Project and Gather Data

#### Scope 
This project will extract data from various sources. Data will be cleaned and transformed before loading into a fact and dimension tables. Users will be able to perform analysis by extracting data from the fact and dimension tables. 

Apache spark will be used in this project.

#### Describe and Gather Data 
<ol> 
<li> I94 Immigration Data: This data comes from the US National Tourism and Trade Office. A data dictionary is included in the workspace. This is where the data comes from. There's a sample file so you can take a look at the data in csv format before reading it all in. You do not have to use the entire dataset, just use what you need to accomplish the goal you set at the beginning of the project. </li>
<li> World Temperature Data: This dataset came from Kaggle. You can read more about it here.  </li>
<li> U.S. City Demographic Data: This data comes from OpenSoft. You can read more about it here.  </li>
<li> Airport Code Table: This is a simple table of airport codes and corresponding cities. It comes from here. </li>
<li> Visa type: This is a simple table of visa codes and corresponding visa descriotion. It comes from I94_SAS_Labels_Description.SAS </li>
</ol> 

####  Configure data sources

In [3]:
# Read in the data here
airport_data           = './data_sources/airport-codes_csv.csv'
immigration_data       = "./data_sources/immigration_data_sample.csv"
us_cities_demographics = "./data_sources/us-cities-demographics.csv"
visa_type              = "./data_sources/visa.csv"

#df_immigration_data       = spark.read.format('csv').options(header='true').load( immigration_data )
df_immigration_data       = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')
df_airport_codes          = spark.read.format('csv').options(header='true').load( airport_data )
df_us_cities_demographics = spark.read.format('csv').options(header='true' , delimiter=';').load( us_cities_demographics )
df_visa_type              = spark.read.format('csv').options(header='true').load( visa_type )

#write to parquet
#i94_2016_df.write.parquet("sas_data")
#i94_2016_df=spark.read.parquet("sas_data")

#### Display schema

In [4]:
df_airport_codes.printSchema()
df_immigration_data.printSchema()
df_us_cities_demographics.printSchema()
df_visa_type.printSchema()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

##### Explore immigrants data 

In [5]:
df_immigration_data.limit(10).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,None,NaN,...,U,None,1979.0,10282016,None,None,None,1.897628e+09,None,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,None,1991.0,D/S,M,None,None,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,None,M,1961.0,09302016,M,None,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,None,M,1988.0,09302016,None,None,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,None,M,2012.0,09302016,None,None,AA,9.246846e+10,00199,B2
5,18.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MI,20555.0,...,None,M,1959.0,09302016,None,None,AZ,9.247104e+10,00602,B1
6,19.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NJ,20558.0,...,None,M,1953.0,09302016,None,None,AZ,9.247140e+10,00602,B2
7,20.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NJ,20558.0,...,None,M,1959.0,09302016,None,None,AZ,9.247161e+10,00602,B2
8,21.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NY,20553.0,...,None,M,1970.0,09302016,None,None,AZ,9.247080e+10,00602,B2
9,22.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NY,20562.0,...,None,M,1968.0,09302016,None,None,AZ,9.247849e+10,00608,B1


##### Remove rows with missing values in i94port, i94addr, i94visa and gender

In [6]:
# immigration_data_pd = immigration_data_pd.dropna(how="any", subset=["i94port", "i94addr", "i94visa", "visatype" , "gender"]) 
df_immigration_data = df_immigration_data.dropna(how="any", subset=["i94port", "i94addr", "i94visa", "visatype" , "gender"])

In [7]:
# Create list of valid state from df_us_cities_demographics
valid_states = df_us_cities_demographics.toPandas()["State Code"].unique()
print(valid_states)

# Create user defined function to validate 'state' data
@udf(StringType())
def validate_state(n): 
    """ check for US states """
    if n in valid_states: 
        return n
    return 'other'

['MD' 'MA' 'AL' 'CA' 'NJ' 'IL' 'AZ' 'MO' 'NC' 'PA' 'KS' 'FL' 'TX' 'VA' 'NV'
 'CO' 'MI' 'CT' 'MN' 'UT' 'AR' 'TN' 'OK' 'WA' 'NY' 'GA' 'NE' 'KY' 'SC' 'LA'
 'NM' 'IA' 'RI' 'PR' 'DC' 'WI' 'OR' 'NH' 'ND' 'DE' 'OH' 'ID' 'IN' 'AK' 'MS'
 'HI' 'SD' 'ME' 'MT']


In [8]:
# Extract data with valid states
df_immigration_data = df_immigration_data.withColumn("i94addr" , validate_state(df_immigration_data.i94addr))
df_immigration_data.limit(5).toPandas()


,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,None,1991.0,D/S,M,None,None,3.736796e+09,00296,F1
1,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,None,M,1961.0,09302016,M,None,OS,6.666432e+08,93,B2
2,27.0,2016.0,4.0,101.0,101.0,BOS,20545.0,1.0,MA,20549.0,...,None,M,1958.0,04062016,M,None,LH,9.247876e+10,00422,B1
3,28.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,MA,20549.0,...,None,M,1960.0,04062016,F,None,LH,9.247890e+10,00422,B1
4,29.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,MA,20561.0,...,None,M,1954.0,09302016,M,None,AZ,9.250378e+10,00614,B2


In [9]:
# Keep US state data ( state != 'other')
df_immigration_data = df_immigration_data.filter(df_immigration_data.i94addr != 'other') 
df_immigration_data.limit(5).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,None,1991.0,D/S,M,None,None,3.736796e+09,00296,F1
1,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,None,M,1961.0,09302016,M,None,OS,6.666432e+08,93,B2
2,27.0,2016.0,4.0,101.0,101.0,BOS,20545.0,1.0,MA,20549.0,...,None,M,1958.0,04062016,M,None,LH,9.247876e+10,00422,B1
3,28.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,MA,20549.0,...,None,M,1960.0,04062016,F,None,LH,9.247890e+10,00422,B1
4,29.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,MA,20561.0,...,None,M,1954.0,09302016,M,None,AZ,9.250378e+10,00614,B2


In [10]:
# group data by city and state
df_immigration_data_clean = df_immigration_data.select(col("cicid").alias("id"), 
                                       col("arrdate").alias("arrdate"),
                                       col("i94port").alias("city_code"),
                                       col("i94addr").alias("state_code"),
                                       col("i94bir").alias("age"),
                                       col("gender").alias("gender"),
                                       col("i94visa").alias("visa_type"),
                                       "count").drop_duplicates()
 

In [11]:
# df_immigration_data_clean.limit(5).toPandas()

In [12]:
# store in staging 
staging_immigration_df = df_immigration_data_clean.select("id", "arrdate" ,"state_code", "city_code", "gender", "age", "visa_type", "count").drop_duplicates()


##### Explore and clean airport data 

In [13]:
#airport_data_pd.head()
df_airport_codes.limit(10).toPandas()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,None,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,None,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450,NA,US,US-AK,Anchor Point,00AK,None,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820,NA,US,US-AL,Harvest,00AL,None,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237,NA,US,US-AR,Newport,None,None,None,"-91.254898, 35.6087"
5,00AS,small_airport,Fulton Airport,1100,NA,US,US-OK,Alex,00AS,None,00AS,"-97.8180194, 34.9428028"
6,00AZ,small_airport,Cordes Airport,3810,NA,US,US-AZ,Cordes,00AZ,None,00AZ,"-112.16500091552734, 34.305599212646484"
7,00CA,small_airport,Goldstone /Gts/ Airport,3038,NA,US,US-CA,Barstow,00CA,None,00CA,"-116.888000488, 35.350498199499995"
8,00CL,small_airport,Williams Ag Airport,87,NA,US,US-CA,Biggs,00CL,None,00CL,"-121.763427, 39.427188"
9,00CN,heliport,Kitchen Creek Helibase Heliport,3350,NA,US,US-CA,Pine Valley,00CN,None,00CN,"-116.4597417, 32.7273736"


#### Clean airports dataset by filter only iso_country = US

#print(airport_data_pd["type"].unique())

In [14]:
#airport_data_pd = airport_data_pd.query('iso_country == "US"')
df_airport_codes_cleaned = df_airport_codes.filter(df_airport_codes["iso_country"] == "US")



#### Clean airports dataset by filter only type = (small / medium / large) airports


In [15]:
#airport_data_pd = airport_data_pd.query('type in ("small_airport","medium_airport","large_airport")')
df_airport_codes_cleaned = df_airport_codes_cleaned.filter( (df_airport_codes_cleaned["type"] == "small_airport") | (df_airport_codes_cleaned["type"]=="medium_airport") | (df_airport_codes_cleaned["type"] == "large_airport") ) 

##### Verify records

In [16]:
df_airport_codes_cleaned.createOrReplaceTempView("df_airport_codes_table")
spark.sql("SELECT distinct type  FROM df_airport_codes_table").show()

+--------------+
|          type|
+--------------+
| large_airport|
|medium_airport|
| small_airport|
+--------------+



In [17]:
staging_airport_df = df_airport_codes_cleaned.select("ident", "type", "name", "continent", "iso_country", "iso_region" , "local_code" ,"coordinates" ).drop_duplicates()


#### Verify US cities demographics data

In [18]:
df_us_cities_demographics.limit(5).toPandas()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601,41862,82463,1562,30908,2.6,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129,49500,93629,4147,32935,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040,46799,84839,4819,8229,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127,87105,175232,5821,33878,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040,143873,281913,5829,86253,2.73,NJ,White,76402


In [19]:
staging_demographics_df = df_us_cities_demographics.select("City", "State", "Median Age" , "Male Population" , "Female Population" , "Total Population" , "State Code" ,"Race" ,"Count").drop_duplicates()

In [20]:
# staging_demographics_df.limit(10).toPandas()

#### Verify Visa type data

In [21]:
df_visa_type.limit(5).toPandas()

,visa_code,visa
0,1,Business
1,2,Pleasure
2,3,Student


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

##### Staging Table
<ul>
<li> staging_immigration_df 
     <ul><li> id, arrdate, state_code, city_code, gender, age, visa_type, count </li></ul></li>
<li> staging_airport_df 
     <ul><li> ident, type, name, continent, iso_country, iso_region, local_code, coordinates</li></ul></li>
<li>staging_demographics_df
    <ul><li> City, State, Median Age, Male Populaiton, Female Populaiton, Total Population, City Code, State Code, Race, Count</li></ul></li>
</ul>     
       
##### Star Schema
###### Fact Table
<ul>
<li>immigration_fact
    <ul><li>arrdate, state_code, city_code, visa_type, count</li></ul>
</li>
</ul>     
    
###### Dimension Table
<ul>
<li> airport    
     <ul><li> ident, type, name, continent, iso_country, iso_region, local_code, coordinates </li></ul></li>  
<li> demographics
    <ul> <li> City, State, Median Age, Male Populaiton, Female Populaiton, Total Population, City Code, State Code, Race</li> </ul> </li> 
<li> visa    
    <ul><li>visa_code, visa </li> </ul></li>   
<li> immigrant
    <ul><li>id, gender, age, visa_type</li></ul></li>
</ul>    

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model
<ol>
    <li> Data cleaning</li>
    <li> Load data into staging tables </li>
    <li> Create and load data into fact and dimensions table</li></ol>

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [22]:
# Fact table

In [23]:
immigration_fact = staging_immigration_df.select("state_code", "city_code", "visa_type", "count").drop_duplicates()

In [24]:
# Dimensions table

In [25]:
airport = staging_airport_df.select("ident", "type", "name", "continent", "iso_country", "iso_region" , "local_code" ,"coordinates" ).drop_duplicates()

In [26]:
demographics = staging_demographics_df.select("City", "State", "Median Age" , "Male Population" , "Female Population" , "Total Population" , "State Code" ,"Race").drop_duplicates()

In [27]:
visa = df_visa_type.select("visa_code", "visa").drop_duplicates()

In [28]:
immigrant = staging_immigration_df.select("id", "gender", "age", "visa_type").drop_duplicates()

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [29]:
# Create user defined function to check for tables exist

In [30]:
def check_tables (df , table_name):
    """check for table exists and return if data quality checks (table exist) has passed"""
    if df is not None:
        print("data quality checks (table exist) passed for {}".format(table_name))  
    else: 
        print("data quality checks (table exist) did not pass for table {}".format(table_name))

In [31]:
# check immigration (fact table)

In [32]:
check_tables(immigration_fact, "immigration_fact_table")

data quality checks (table exist) passed for immigration_fact_table


In [33]:
# check dimensions table

In [34]:
check_tables(airport, "airport")  

data quality checks (table exist) passed for airport


In [35]:
check_tables(demographics, "demographics")  

data quality checks (table exist) passed for demographics


In [36]:
check_tables(visa, "visa") 

data quality checks (table exist) passed for visa


In [37]:
check_tables(immigrant, "immigrant")

data quality checks (table exist) passed for immigrant


In [38]:
# Create user defined function to check for number of data rows

In [39]:
def check_rows(df , table_name):
    """check for number of rows in each table and return if data quality checks (for rows) has passed"""
    result = df.count()  
    if ( result != 0 ) : 
        print("data quality checks(number of rows) passed for {} with {} rows".format(table_name, result))  
    else: 
        print("data quality checks(number of rows) did not pass for table {}".format(table_name))

In [40]:
# check immigration (fact table)

In [41]:
check_rows(immigration_fact, "immigration_fact_table")

data quality checks(number of rows) passed for immigration_fact_table with 8688 rows


In [42]:
# check dimensions table

In [43]:
check_rows(airport, "airport")  

data quality checks(number of rows) passed for airport with 14582 rows


In [44]:
check_rows(demographics, "demographics")  

data quality checks(number of rows) passed for demographics with 2891 rows


In [45]:
check_rows(visa, "visa") 

data quality checks(number of rows) passed for visa with 3 rows


In [46]:
check_rows(immigrant, "immigrant")

data quality checks(number of rows) passed for immigrant with 2435922 rows


#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

##### Fact Table
 
immigration_fact
<ul> 
    <li>arrdate: arrival date </li>
    <li>state_code: state code of arrival city</li>
    <li>city_code: city code of arrival city</li>
    <li>visa_type: immigrant's visa type</li>
    <li>count: number of immigrant entering US</li>
</ul>


##### Dimension Table

airport
<ul> 
    <li>ident: id of airport </li>
    <li>type : airport type </li>
    <li>name : name of airport </li>
    <li>continent: continent of airport </li>
    <li>iso_country: country of airport </li>
    <li>iso_region: region of airport </li>
    <li>local_code: local code of airport </li>
    <li>coordinates : coordinates of airport </li>
</ul>

  
demographics
<ul> 
    <li>City : city name </li>
    <li>State : state name </li>
    <li>Median Age : median age of city</li>
    <li>Male Populaiton : male population in city  </li>
    <li>Female Populaiton : female population in city </li> 
    <li>Total Population : total population in city </li>
    <li>City Code : city code </li>
    <li>State Code : state code </li>
    <li>Race: race of the population </li>
</ul>

        
visa
<ul> 
    <li>visa_code: code of the visa issued</li>
    <li>visa: name of the visa issued </li>
</ul>
    
    
immigrant
<ul> 
    <li>id: id of immigrant</li>
    <li>gender: gender of immigrant </li>
    <li>age: age of immigrant </li>
    <li>visa_type: visa typed of immigrant  </li>
</ul>


#### Step 5: Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

##### Rationale for the choice of tools and technologies for the project. 

Apache Spark was chosen for its ability to process big data in a distributed manner and it also utilizes in-memory caching to optimized query execution. 
In addition, it has libraries that enable users to further perform data transaformation to serve other needs.

##### Data update frequency
Data should be updated based on users' reporting needs. If the data can be made available on a daily basis and users have to analyse the data on a daily basis, the data should be updated on daily manner. 

Otherwise, it is recommended to perform an update on monthly basis. 


##### Consider the following scenarios
<ul> 
    <li>
If data was increased by 100x, there might be performance issues when running on users' machine. Users should consider setting up Amazon instances to host the apache spark because users' machine may not have sufficient RAM to process data.  </li>
  <li>
If data populates a dashboard that must be updated on a daily basis by 7am every day, users should consider using Airflow dags to schedule and automate the data pipeline, and set rules to inform users if the dashboard is updated with latest data at 7am. If data is not available by 7am, another rule should be triggered to run the pipeline again so that the dashboard can be refreshed.  </li>
  <li>
If the database needs to be accessed by 100+ people, users should consider hosting a data warehouse (e.g. Amazon redshift cluster) in the cloud as the data availability and accessibility can be guaranteed by the provider's service level agreement.  </li>
 </ul> 